## DL EH Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'eh'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        print('split type',type(y))
        if isinstance(y, pd.DataFrame):
            y = y.values[:,0]
        elif isinstance(y, pd.Series):
            try:
                y = y.values[:,0]
            except:
                y = y.values
        elif isinstance(y, np.ndarray):
            try:
                if y.shape[1]>1:
                    y = y[:,0]
            except:
                pass

        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'eh'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()
    #y = pd.concat([y,y], axis=1)
    #y.columns = ['target1', 'target2']
    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
split type <class 'pandas.core.series.Series'>
0
model: eh
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
integration_values.shape[0] 132116
integration_values.shape[0] 30266
Concordance Index 0.6180508709971827
Integrated Brier Score: 0.17396428065972153
{'model': 'eh', 'dataset': 'METABRIC', 'cindex_train': [0.6669823129795596], 'cindex_test': [0.6180508709971827], 'ibs_train': [0.1637659244810419], 'ibs_score_test': [0.17396428065972153]}
1
model: eh
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
integration_values.shape[0] 153177
integration_values.shape[0] 34102
Concordance Index 0.6441817931522744
Integrated Brier Score: 0.17109570398349827
{'model': 'eh', 'dataset': 'METABRIC', 'cindex_train': [0.6694577636929593], 'cindex_test': [0.6441817931522744], 'ibs_train': [0.16187699198442543], 'ibs_score_test': [0.17109570398349827]}
2
model: eh

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,eh,METABRIC,0.666982,0.618051,0.163766,0.173964
1,eh,METABRIC,0.669458,0.644182,0.161877,0.171096
2,eh,METABRIC,0.673020,0.643062,0.161801,0.174084
3,eh,METABRIC,0.673285,0.659609,0.159776,0.165943
4,eh,METABRIC,0.659962,0.659924,0.164809,0.170034
5,eh,FLCHAIN,0.765179,0.799582,0.100106,0.096730
6,eh,FLCHAIN,0.763320,0.786669,0.099407,0.098253
7,eh,FLCHAIN,0.777486,0.772324,0.097700,0.100715
8,eh,FLCHAIN,0.777179,0.760953,0.097805,0.101137
9,eh,FLCHAIN,0.769819,0.790232,0.099446,0.093953


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'ah'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
split type <class 'pandas.core.series.Series'>
0
model: ah
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>


ValueError: Expected value argument (Tensor of shape (52, 23)) to be within the support (Real()) of the distribution Normal(loc: 0.0, scale: 1.0), but found invalid values:
tensor([[ 5.3703e+01,  6.6545e+01, -5.9238e+01,  ...,  9.8671e+01,
          3.2243e+01,  6.6515e+01],
        [ 0.0000e+00,  1.2841e+01, -1.1294e+02,  ...,  4.4968e+01,
         -2.1460e+01,  1.2811e+01],
        [-7.9723e+01, -6.6882e+01, -1.9266e+02,  ..., -3.4756e+01,
         -1.0118e+02, -6.6912e+01],
        ...,
        [-1.2811e+01,  2.9929e-02, -1.2575e+02,  ...,  3.2156e+01,
         -3.4272e+01,  0.0000e+00],
        [ 8.1151e+01,  9.3992e+01, -3.1791e+01,  ...,  1.2612e+02,
          5.9690e+01,  9.3962e+01],
        [-2.1058e+00,  1.0735e+01, -1.1505e+02,  ...,  4.2862e+01,
         -2.3566e+01,  1.0705e+01]])

In [ ]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)